In [2]:
import pygame
import heapq
import sys
import time

# Initialize pygame
pygame.init()

# Initialize the mixer for sound effects and music
pygame.mixer.init()

# Define colors and settings
WALL_COLOR = (0, 0, 0)
PATH_COLOR = (0, 255, 0)
START_COLOR = (0, 0, 255)
GOAL_COLOR = (255, 0, 0)
OPEN_COLOR = (255, 255, 255)
COIN_COLOR = (255, 215, 0)
LAVA_COLOR = (255, 69, 0)
PLAYER_COLOR = (0, 0, 255)
HIGHLIGHT_COLOR = (0, 0, 255)  # Blue color for hover effect
GOAL_REACHED_COLOR = (0, 255, 0)  # Color when the goal is reached

# Load sounds
start_sound = pygame.mixer.Sound('start_sound.mp3')
coin_sound = pygame.mixer.Sound('coin_sound.mp3')
goal_sound = pygame.mixer.Sound('goal_sound.mp3')

# Load background music
try:
    pygame.mixer.music.load('background_music.mp3')
except pygame.error as e:
    print(f"Error loading background music: {e}")
    sys.exit(1)

# Game history to keep track of the last 5 games
game_history = []

def parse_maze(file_path):
    """Reads the maze from a file and returns it as a list of lists."""
    try:
        with open(file_path, 'r') as file:
            maze = [list(line.strip()) for line in file]
        return maze
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        sys.exit(1)
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        sys.exit(1)

def find_start_goal(maze):
    """Finds the start and goal positions in the maze."""
    start = goal = None
    for r in range(len(maze)):
        for c in range(len(maze[0])):
            if maze[r][c] == 'S':
                start = (r, c)
            elif maze[r][c] == 'G':
                goal = (r, c)
    return start, goal

def heuristic(a, b):
    """Heuristic function for A* algorithm: Manhattan distance."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def neighbors(maze, node):
    """Returns all valid neighboring nodes for a given node, including diagonal moves."""
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    result = []
    for d in directions:
        nr, nc = node[0] + d[0], node[1] + d[1]
        if 0 <= nr < len(maze) and 0 <= nc < len(maze[0]) and maze[nr][nc] != 'X':
            result.append((nr, nc))
    return result

def a_star_search(maze, start, goal):
    """A* search algorithm to find the shortest path."""
    pq = []
    heapq.heappush(pq, (0, start, 0, 0, []))  # (priority, current, cost, coins_collected, path)
    visited = set()
    while pq:
        current_priority, current, cost, coins_collected, path = heapq.heappop(pq)
        if current in visited:
            continue
        path.append(current)
        visited.add(current)

        if current == goal:
            time_complexity = len(visited)
            space_complexity = len(pq) + len(visited)
            return path, coins_collected, time_complexity, space_complexity

        for neighbor in neighbors(maze, current):
            if neighbor in visited or maze[neighbor[0]][neighbor[1]] == 'X':
                continue
            extra_cost = int(maze[neighbor[0]][neighbor[1]]) if maze[neighbor[0]][neighbor[1]].isdigit() else 0
            new_cost = cost + 1
            new_coins = coins_collected + extra_cost
            if extra_cost > 0:
                coin_sound.play()
            heapq.heappush(pq, (new_cost + heuristic(neighbor, goal), neighbor, new_cost, new_coins, path.copy()))

    return None, 0, len(visited), len(pq) + len(visited)

def bfs_search(maze, start, goal):
    """Breadth-First Search algorithm to find the shortest path in terms of steps."""
    queue = [(start, 0, 0, [])]  # (current, cost, coins_collected, path)
    visited = set()
    while queue:
        current, cost, coins_collected, path = queue.pop(0)
        if current in visited:
            continue
        path.append(current)
        visited.add(current)

        if current == goal:
            time_complexity = len(visited)
            space_complexity = len(queue) + len(visited)
            return path, coins_collected, time_complexity, space_complexity

        for neighbor in neighbors(maze, current):
            if neighbor in visited or maze[neighbor[0]][neighbor[1]] == 'X':
                continue
            extra_cost = int(maze[neighbor[0]][neighbor[1]]) if maze[neighbor[0]][neighbor[1]].isdigit() else 0
            new_coins = coins_collected + extra_cost
            if extra_cost > 0:
                coin_sound.play()
            queue.append((neighbor, cost + 1, new_coins, path.copy()))

    return None, 0, len(visited), len(queue) + len(visited)

def dijkstra_search(maze, start, goal):
    """Dijkstra's algorithm to find the shortest path."""
    pq = []
    heapq.heappush(pq, (0, start, 0, []))  # (cost, current, coins_collected, path)
    visited = set()
    while pq:
        current_cost, current, coins_collected, path = heapq.heappop(pq)
        if current in visited:
            continue
        path.append(current)
        visited.add(current)

        if current == goal:
            time_complexity = len(visited)
            space_complexity = len(pq) + len(visited)
            return path, coins_collected, time_complexity, space_complexity

        for neighbor in neighbors(maze, current):
            if neighbor in visited or maze[neighbor[0]][neighbor[1]] == 'X':
                continue
            extra_cost = int(maze[neighbor[0]][neighbor[1]]) if maze[neighbor[0]][neighbor[1]].isdigit() else 0
            new_coins = coins_collected + extra_cost
            if extra_cost > 0:
                coin_sound.play()
            heapq.heappush(pq, (current_cost + 1, neighbor, new_coins, path.copy()))

    return None, 0, len(visited), len(pq) + len(visited)

def draw_maze(screen, maze, path=None, coins_collected=0):
    """Draws the maze using Pygame and highlights the path found."""
    rows, cols = len(maze), len(maze[0])
    screen_width, screen_height = screen.get_size()
    cell_size = min(screen_width // (cols + 2), screen_height // (rows + 2))  # Adjust for lava border
    font_size = max(12, cell_size // 2)  # Half the cell size or 12, whichever is larger

    font = pygame.font.Font(None, font_size)  # Create font with dynamic sizing

    screen.fill(LAVA_COLOR)  # Background color for the maze area

    def draw_lava_border():
        """Draws a border of lava around the maze."""
        for r in range(rows + 2):
            for c in range(cols + 2):
                if r == 0 or r == rows + 1 or c == 0 or c == cols + 1:
                    if not ((r == 0 and c == goal[1] + 1) or (r == rows + 1 and c == start[1] + 1)):
                        rect = pygame.Rect(c * cell_size, r * cell_size, cell_size, cell_size)
                        pygame.draw.rect(screen, WALL_COLOR, rect)
                else:
                    rect = pygame.Rect(c * cell_size, r * cell_size, cell_size, cell_size)
                    pygame.draw.rect(screen, LAVA_COLOR, rect)

    draw_lava_border()

    for r in range(rows):
        for c in range(cols):
            color = OPEN_COLOR  # Default color
            if maze[r][c] == 'X':
                color = WALL_COLOR
            elif (r, c) == start:
                color = START_COLOR
            elif (r, c) == goal:
                color = GOAL_COLOR
            elif maze[r][c].isdigit() and maze[r][c] != '0':
                color = COIN_COLOR  # Coin cells

            rect = pygame.Rect((c + 1) * cell_size, (r + 1) * cell_size, cell_size, cell_size)
            pygame.draw.rect(screen, color, rect)

            # Draw text for goal and coin cells
            if (r, c) == goal:
                text = font.render('G', True, (0, 0, 0), color)
                text_rect = text.get_rect(center=rect.center)
                screen.blit(text, text_rect)
            elif maze[r][c].isdigit():
                text = font.render(maze[r][c], True, (0, 0, 0), color)
                text_rect = text.get_rect(center=rect.center)
                screen.blit(text, text_rect)

    if path:
        for (r, c) in path:
            rect = pygame.Rect((c + 1) * cell_size, (r + 1) * cell_size, cell_size, cell_size)
            pygame.draw.rect(screen, PATH_COLOR, rect)
            if maze[r][c].isdigit() and maze[r][c] != '0':
                text = font.render(maze[r][c], True, (0, 0, 0), PATH_COLOR)
                text_rect = text.get_rect(center=rect.center)
                screen.blit(text, text_rect)
            elif (r, c) == goal:
                pygame.draw.rect(screen, GOAL_COLOR, rect)
                text = font.render('G', True, (0, 0, 0), GOAL_COLOR)
                text_rect = text.get_rect(center=rect.center)
                screen.blit(text, text_rect)

    # Draw the 'S' for the start position in black color after the path is drawn
    start_rect = pygame.Rect((start[1] + 1) * cell_size, (start[0] + 1) * cell_size, cell_size, cell_size)
    pygame.draw.rect(screen, START_COLOR, start_rect)
    text = font.render('S', True, (0, 0, 0))
    text_rect = text.get_rect(center=start_rect.center)
    screen.blit(text, text_rect)

    pygame.display.update()

def animate_player(screen, maze, path, cell_size, coins_collected):
    """Animates the player moving along the path."""
    running = True
    paused = False
    current_index = 0
    goal_reached = False
    speed = 0.1  # Initial speed

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    main()
                    return
                elif event.key == pygame.K_q:
                    running = False
                elif event.key == pygame.K_h:
                    display_history(screen, game_history)
                elif event.key == pygame.K_UP:
                    speed = max(0.01, speed - 0.01)  # Increase speed (lower delay)

        if not paused and current_index < len(path):
            pos = path[current_index]
            r, c = pos
            if pos == path[-1]:
                goal_reached = True
                paused = True  # Pause the game when the goal is reached
            rect = pygame.Rect((c + 1) * cell_size, (r + 1) * cell_size, cell_size, cell_size)
            pygame.draw.rect(screen, GOAL_REACHED_COLOR if goal_reached else PLAYER_COLOR, rect)
            pygame.display.update()
            time.sleep(speed)  # Adjust the delay for animation effect
            current_index += 1
            draw_maze(screen, maze, path=path[:current_index], coins_collected=coins_collected)  # Redraw the path up to current position

        if goal_reached:
            font = pygame.font.Font(None, 36)
            text = font.render(f"Total Coins Collected: {coins_collected}", True, (255, 255, 255))
            restart_text = font.render("Press R to Restart, Q to Quit, H for History", True, (255, 255, 255))
            screen.blit(text, (10, 10))
            screen.blit(restart_text, (10, 50))
            pygame.display.flip()

def display_history(screen, history):
    """Displays the game history."""
    font = pygame.font.Font(None, 36)
    small_font = pygame.font.Font(None, 28)
    screen.fill((0, 0, 0))
    title = font.render("Game History", True, (255, 255, 255))
    screen.blit(title, (50, 50))

    if not history:
        no_history_text = small_font.render("No history available.", True, (255, 255, 255))
        screen.blit(no_history_text, (50, 100))
    else:
        for i, game in enumerate(history):
            algo, maze_size, moves, coins, time_complexity, space_complexity = game
            algo_name = "A*" if algo == "a_star" else "Dijkstra" if algo == "dijkstra" else "BFS"
            history_text = small_font.render(f"{i + 1}. Algo: {algo_name}, Maze Size: {maze_size}, Moves: {moves}, Coins: {coins}", True, (255, 255, 255))
            history_text2 = small_font.render(f"Time Complexity: O({time_complexity}), Space Complexity: O({space_complexity})", True, (255, 255, 255))
            screen.blit(history_text, (50, 100 + i * 80))
            screen.blit(history_text2, (50, 130 + i * 80))

    back_text = font.render("Press B to go back to main menu", True, (255, 255, 255))
    screen.blit(back_text, (50, 100 + len(history) * 80 + 50))

    pygame.display.flip()
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN or event.key == pygame.K_b:
                    waiting = False
                elif event.key == pygame.K_q:
                    pygame.quit()
                    sys.exit()
    main()  # Go back to the main menu

def main():
    global start, goal, maze, game_history

    # Check if mixer is initialized, if not, initialize it
    if not pygame.mixer.get_init():
        pygame.mixer.init()

    # Initial welcome screen to choose maze size
    screen = pygame.display.set_mode((800, 800), pygame.RESIZABLE)
    pygame.display.set_caption("Maze Solver")

    font = pygame.font.Font(None, 50)
    small_font = pygame.font.Font(None, 36)
    screen.fill((0, 0, 0))
    text = font.render("Choose Maze Size", True, (255, 255, 255))
    options = ["1. 11x11", "2. 31x31", "3. 101x101"]
    screen.blit(text, (200, 300))
    for i, option in enumerate(options):
        option_text = small_font.render(option, True, (255, 255, 255))
        screen.blit(option_text, (250, 400 + i * 50))
    pygame.display.flip()

    selected_index = 0
    selecting_maze_size = True
    while selecting_maze_size:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_DOWN:
                    selected_index = (selected_index + 1) % 3
                    screen.fill((0, 0, 0))
                    screen.blit(text, (200, 300))
                    for i, option in enumerate(options):
                        color = HIGHLIGHT_COLOR if i == selected_index else (255, 255, 255)
                        option_text = small_font.render(option, True, color)
                        screen.blit(option_text, (250, 400 + i * 50))
                    pygame.display.flip()
                elif event.key == pygame.K_UP:
                    selected_index = (selected_index - 1) % 3
                    screen.fill((0, 0, 0))
                    screen.blit(text, (200, 300))
                    for i, option in enumerate(options):
                        color = HIGHLIGHT_COLOR if i == selected_index else (255, 255, 255)
                        option_text = small_font.render(option, True, color)
                        screen.blit(option_text, (250, 400 + i * 50))
                    pygame.display.flip()
                elif event.key == pygame.K_RETURN:
                    maze_files = ["maze_11x11.txt", "maze_31x31.txt", "maze_101x101.txt"]
                    maze_file = maze_files[selected_index]
                    selecting_maze_size = False

    maze = parse_maze(maze_file)
    start, goal = find_start_goal(maze)

    # Choose algorithm screen
    def draw_algorithm_screen(selected_index):
        options = ["1. A* Search", "2. Dijkstra's Algorithm", "3. Breadth-First Search"]
        screen.fill((0, 0, 0))
        text = font.render("Choose Algorithm", True, (255, 255, 255))
        screen.blit(text, (200, 300))
        for i, option in enumerate(options):
            color = HIGHLIGHT_COLOR if i == selected_index else (255, 255, 255)
            option_text = small_font.render(option, True, color)
            screen.blit(option_text, (250, 400 + i * 50))
        pygame.display.flip()

    selected_index = 0
    draw_algorithm_screen(selected_index)

    selecting_algorithm = True
    while selecting_algorithm:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_DOWN:
                    selected_index = (selected_index + 1) % 3
                    draw_algorithm_screen(selected_index)
                elif event.key == pygame.K_UP:
                    selected_index = (selected_index - 1) % 3
                    draw_algorithm_screen(selected_index)
                elif event.key == pygame.K_RETURN:
                    if selected_index == 0:
                        algorithm = "a_star"
                    elif selected_index == 1:
                        algorithm = "dijkstra"
                    else:
                        algorithm = "bfs"
                    selecting_algorithm = False

    start_sound.play()
    pygame.mixer.music.play(-1)  # Start the background music

    if algorithm == "a_star":
        path, coins_collected, time_complexity, space_complexity = a_star_search(maze, start, goal)
    elif algorithm == "dijkstra":
        path, coins_collected, time_complexity, space_complexity = dijkstra_search(maze, start, goal)
    else:
        path, coins_collected, time_complexity, space_complexity = bfs_search(maze, start, goal)

    moves = len(path) if path else 0
    maze_size = f"{len(maze)}x{len(maze[0])}"
    game_history.append((algorithm, maze_size, moves, coins_collected, time_complexity, space_complexity))
    if len(game_history) > 5:
        game_history.pop(0)

    draw_maze(screen, maze, path, coins_collected)

    animate_player(screen, maze, path, 800 // (len(maze[0]) + 2), coins_collected)

    pygame.mixer.music.stop()  # Stop the background music after the game ends

    # Display the coin score and options to restart or quit
    font = pygame.font.Font(None, 36)
    text = font.render(f"Total Coins Collected: {coins_collected}", True, (255, 255, 255))
    restart_text = small_font.render("Press R to Restart, Q to Quit, H for History", True, (255, 255, 255))
    screen.blit(text, (10, 10))
    screen.blit(restart_text, (10, 50))
    pygame.display.flip()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                elif event.key == pygame.K_r:
                    main()
                elif event.key == pygame.K_h:
                    display_history(screen, game_history)

    pygame.quit()

if __name__ == "__main__":
    main()
